 TO convert data array to Geotiff that is projected to epsg 4326,    #<br>
   i.e. WGS84. To keep the original resolution and keep the output   #<br>
   file size in check, the default orient='slant' option puts the    #<br>
   projected GeoTiff the slanted swath of the original data, instead #<br>
   of the more commonly practiced retangular latitude-longitude grid #<br>
  (epsg 4326) that requires addition of many not-value fill-in       #<br>
   pixels, i.e. upscaling. (There are still some fill-in cells in    #<br>
   maitaining slanted swath, but the upscaling is much reduced       #<br>
   especially when  the selected segment is extended into the high   #<br>
  latitude region.                                                   #<br>

In [ ]:

import numpy as np
from pyproj import Proj, transform
from scipy.interpolate import griddata

def remapSwath(arr,Lon,Lat,latRange, fillv=-9999.9, orient='slant'):
    """
    Remapping Swath data to EPSG4326 projection 
    Input:
      arr: input data array
      (Lon, Lat): array of input data's (lon,lat) location, same dimensions as arr
      fillv: fille value in the data
      latRange: latitude range of the data to be converted to geotiff
      orient: orientation of the output geotiff. Currently only 'slant' works
    
    Output:
    bandArr: remapped arr data array
    GeoTransf: geotransform parameters for geotiff
    """
    # pad data perimeter for nearest neighbor data location
    arrP = arrPadding(arr, fillv)
    LonP,LatP = coordPadding(Lon, Lat)
    
    # get geotransfer parameters 
    GeoTransf, lonProj, latProj = getGeoTransSlant(Lon,Lat,latRange)
    
    # get remapped data array 
    bandArr = projArr(LonP, LatP, arrP, lonProj, latProj)
    
    return bandArr, GeoTransf
  
def projArr(Lon, Lat, arr, lonProj, latProj):
    """Re-grid data to the projeted (lonProj, latProj) grid"""
    points = np.array( (Lon.flatten(), Lat.flatten()) ).T
    values = arr.flatten()
    return griddata( points, values, (lonProj, latProj),  method='nearest' )
  
def arrPadding(arr,fillv,xpd=2):
    """
    Add padding to data array for re-gridding
    xpd is the extra layer(s) of padding
    """
    nyf, nxf  = arr.shape
    arrP = np.full((nyf,nxf+2*xpd),fillv)
    arrP[:,xpd:-xpd] = arr
    
    return arrP
    
def PadLon(LonX):
    nyf, nxf  = LonX.shape
    LonP = np.empty(shape=(nyf,nxf+2))
    LonP[:,1:-1] = LonX
    LonP[:, 0] = 2*LonP[:,1] - LonP[:,2]
    LonP[:,-1] = 2*LonP[:,-2] - LonP[:,-3]
    LonP[LonP<-180]=LonP[LonP<-180]+360
    LonP[LonP>180]=LonP[LonP>180]-180
    
    return LonP
  
def coordPadding(Lon,Lat,xpd=2):
    """
    Add padding to the (lat,lon) arrays for re-gridding
    xpd is the extra layer(s) of padding
    Rain product may need xpd>1 for some cases. So use xpd=3 to be safe. 
    """
    nyf, nxf  = Lon.shape
    LonP = Lon.copy()
    LatP = np.empty(shape=(nyf,nxf+2*xpd))
    LatP[:,xpd:-xpd] = Lat.copy()
    for ix in range(xpd):
        ixn = -ix-1
        LatP[:, ix] = Lat[:,0] + (xpd-ix)*(Lat[:,0]- Lat[:,1])
        LatP[:,ixn] = Lat[:,-1]+ (xpd-ix)*(Lat[:,-1]- Lat[:,-2])
    
        LonP = PadLon(LonP)
        
    return LonP, LatP
  
def dLonMin(Lons):
    """Find minimal longitude span in degrees"""
    widths=[]
    for j in range(Lons.shape[0]):
        widths.append(np.abs(Lons[j,0]-Lons[j,-1]))    
    return min(widths)
        
def getGeoTransSlant(Lonx,Latx,latRange):
    """
    Geotransformation for projection to use regular (lon, lat) EPSG 4326 (WGS84)
    Used for SWATHS data with (lon, lat)
    """
    latRange = [min(latRange),max(latRange)]
    Lon =Lonx.copy()
    Lat =Latx.copy()
    nyf, nxf  = Lon.shape
    nxc = int(nxf/2)
    lats = Lat[:,nxc]
    j1 = np.abs(lats-latRange[0]).argmin() #<-- the lower latitude
    j2 = np.abs(lats-latRange[1]).argmin() #<-- the higher latitude

    #---for mixed +/- longitudes in data location
    (ja,jb)=(j1,j2) if(j2>j1) else (j2,j1)
    Lonmin, Lonmax = Lon[ja:jb+1,:].min(), Lon[ja:jb+1,:].max()
    if(np.all(Lon[ja:jb+1,:]>=0) or np.all(Lon[ja:jb+1,:]<=0) or (Lonmin>-40 or Lonmax<40)):
        pass
    else:
        indx = Lon<0
        Lon[indx] = Lon[indx] + 360.
                
    #--Note that projected img start from upper-Left downwards
    Long = lats[j1] - lats[j2]
    Broad = Lon[j1,nxc] - Lon[j2,nxc]      #<-- sign dep. on Asc or Dsc

    #---To find the slant angle, get delta(Lon) and delta(Lat) first
    #  Only 2 conditions for AMSR, Ascend or Descend mode
    #  W is always "+" (it's projected)
    #  Del is "+" for Asc (SE -> NW) & Lat[j,0] - Lat[j,-1] is "+", so it works
    #  Del is "-" for Dsc (NE -> SW) & Lat[j,0] - Lat[j,-1] is "-", so it works
    dlon1 = np.abs(Lon[j1,0]-Lon[j1,-1])
    dlon2 = np.abs(Lon[j2,0]-Lon[j2,-1])
    
    if dlon1 > dlon2:
        Width = dlon1
        Del = Lat[j1,0] - Lat[j1,-1]
    else:
        Width = dlon2
        Del = Lat[j2,0] - Lat[j2,-1]
        
    LonSubset = Lon[j1:j2+1,:] if(j2 > j1) else Lon[j2:j1+1, :]
    degRat = Width/dLonMin( LonSubset )
    Ny = abs(j2 -j1) +1
    Nx = int(nxf*degRat)+1
    
    #--- Constructing affine geotransform parameters
    # Xgeo = g0 + Xpix*g1 + Yline*g2
    # Ygeo = g3 + Xpix*g4 + Yline*g5
    # Note that the pixel/line coord. in the above are from (0,0) at the top-left corner 
    # of the top-left pixel to (width_in_pixels,height_in_pixels) at the bottom-right
    # corner of the bottom-right pixel. 
    # The enter of the top-left pixel therefore is (0.5,0.5) in pix/line coord.
    # We USE (x0,y0) as the coord. of the center of the top-left pixel
    #    and (g0,g3) as the coord. of the corner of the top-left pixel 
    x0 = Lon[j2,nxc] - Width/2 
    y0 = Lat[j2,nxc] - Del/2
    if(x0>180): x0 = x0-360 #<return to (-180 to 180 range)
    
    #g0, g3 = x0, y0
    g1, g4 = Width/(Nx-1),  Del/(Nx-1)
    g2, g5 = Broad/(Ny-1), Long/(Ny-1)
    g0 = x0 -0.5*g1 -0.5*g2
    g3 = y0 -0.5*g4 -0.5*g5
    
    GeoTransf = (g0, g1, g2, g3, g4, g5)

    #--- Get (lat, lon) for the remapped/regridded swath
    lonProj = np.empty(shape=(Ny,Nx))
    latProj = np.empty(shape=(Ny,Nx))
    for j in range (Ny):
        lonProj[j,:] = x0 + np.arange(Nx)*g1 + j*g2
        latProj[j,:] = y0 + np.arange(Nx)*g4 + j*g5
        
    inds1 = lonProj<-180
    inds2 = lonProj> 180
    lonProj[inds1] = lonProj[inds1] + 360.
    lonProj[inds2] = lonProj[inds2] - 360.
    
    return GeoTransf, lonProj, latProj
